## QR Decomposition


In [1]:
import numpy as np

### QR decomposition using Gram-Schmidt

In [2]:
a1 = np.array([1,0,1,1])
a2 = np.array([0,2,0,3])
a3 = np.array([-3,-1,1,5])
A = np.array([a1,a2,a3]).T

1. Step: Use Gram-Schmidt to find an orthonormal basis of the column space of A
Remark: I do this here 'by hand' but there are many 'automated' version of Gram-Schmidt written in python. However, remember that Gram-Schmidt is very unstable numerically! Therefore in application where you want to find an orthonormal basis of the space spanned by some given vectors one should (from a numerical perspective) rather use the QR decomposition via Householder transformation to obtain such a basis (it is in the Q!)

In [3]:
# Finding an orthogonal basis
v1 = a1 
v2 = a2 - np.dot(v1,a2)/np.dot(v1,v1)*v1
v3 = a3 - np.dot(v1,a3)/np.dot(v1,v1)*v1 - np.dot(v2,a3)/np.dot(v2,v2)*v2

print("v1= ", v1, "\nv2= ",v2,"\nv3= ",v3)

v1=  [1 0 1 1] 
v2=  [-1.  2. -1.  2.] 
v3=  [-3. -3.  1.  2.]


In [4]:
# Normalizing the basis
v1_norm = np.sqrt(np.dot(v1,v1))
v2_norm = np.sqrt(np.dot(v2,v2))
v3_norm = np.sqrt(np.dot(v3,v3))

w1 = 1/v1_norm*v1
w2 = 1/v2_norm*v2
w3 = 1/v3_norm*v3

print("w1= ",w1,"\nw2= ",w2,"\nw3= ", w3)

w1=  [0.57735027 0.         0.57735027 0.57735027] 
w2=  [-0.31622777  0.63245553 -0.31622777  0.63245553] 
w3=  [-0.62554324 -0.62554324  0.20851441  0.41702883]


For comparison with the lecture notes: 

In [5]:
print("1/np.sqrt(3): ", 1/np.sqrt(3))
print("1/np.sqrt(10): ", 1/np.sqrt(10))
print("1/np.sqrt(23): ", 1/np.sqrt(23))

1/np.sqrt(3):  0.5773502691896258
1/np.sqrt(10):  0.31622776601683794
1/np.sqrt(23):  0.20851441405707477


2. Step: Construct matrices Q and R 

Note: Q and R here are named Q_1 and R_1 in the notes

In [6]:
# Construct the matrix Q

Q = np.array([w1,w2,w3]).T

print("Q: \n",Q)

Q: 
 [[ 0.57735027 -0.31622777 -0.62554324]
 [ 0.          0.63245553 -0.62554324]
 [ 0.57735027 -0.31622777  0.20851441]
 [ 0.57735027  0.63245553  0.41702883]]


There are two ways to get the matrix R: 
a) construct it as it is defined in the notes 
b) use that Q is orthogonal matrix and construct it via multiplication of Q.T@A

In [7]:
# Construct the matrix R (version a)
Ra = np.array([[np.dot(w1,a1),np.dot(w1,a2),np.dot(w1,a3)],[np.dot(w2,a1),np.dot(w2,a2),np.dot(w2,a3)],[np.dot(w3,a1),np.dot(w3,a2),np.dot(w3,a3)]])

print("a) Matrix R: \n", Ra)

# Construct the matrix R (version b)
Rb = Q.T@A

print("b) Matrix Rb: \n", Rb)

a) Matrix R: 
 [[ 1.73205081e+00  1.73205081e+00  1.73205081e+00]
 [ 0.00000000e+00  3.16227766e+00  3.16227766e+00]
 [-5.55111512e-17  0.00000000e+00  4.79583152e+00]]
b) Matrix Rb: 
 [[ 1.73205081e+00  1.73205081e+00  1.73205081e+00]
 [ 0.00000000e+00  3.16227766e+00  3.16227766e+00]
 [-5.55111512e-17 -1.11022302e-16  4.79583152e+00]]


In [8]:
# Check that we did everything correct 

print("QRa: \n",Q@Ra)

QRa: 
 [[ 1.00000000e+00  2.76921245e-16 -3.00000000e+00]
 [ 3.47246255e-17  2.00000000e+00 -1.00000000e+00]
 [ 1.00000000e+00  2.76921245e-16  1.00000000e+00]
 [ 1.00000000e+00  3.00000000e+00  5.00000000e+00]]


In [9]:
# Check that we did everything correct 

print("QRb: \n",Q@Rb)

QRb: 
 [[ 1.00000000e+00  2.05937157e-16 -3.00000000e+00]
 [ 3.47246255e-17  2.00000000e+00 -1.00000000e+00]
 [ 1.00000000e+00  1.13338156e-16  1.00000000e+00]
 [ 1.00000000e+00  3.00000000e+00  5.00000000e+00]]


In [10]:
# More numerical and not visual check of this: 

if np.abs((A - Q@Ra).sum()) < 1e-9 and np.abs((A - Q@Rb).sum()) < 1e-9:
    print("true")
else:
    print("false")

true


### QR-decomposition using Householder transformation 

scipy.linalg has a command for this scipy.linalg.qr(A) however we will again first do it by hand.

Note that we will work here with column vectors a1, a2 and a3  and not with row vectors as in the Gram-Schmidt case

1. Step: Construct $\alpha_1$ and $u_1$ and find the elementary reflector $H_1$ for the first transformation

In [11]:
a1 = A[:,[0]]
a1_norm = np.sqrt(sum(a1**2)) 
alpha1 = -np.sign(a1[0])*a1_norm
print("alpha1 = \n", alpha1)
e1=np.zeros_like(a1)
e1[0] = 1
u1 = a1 - alpha1*e1
print("u1 = \n", u1)
I = np.diag([1,1,1,1])
u1_norm = np.sqrt(sum(u1**2))
H1 = I - 2/u1_norm**2*u1@u1.T
print("H1 = \n", H1)

alpha1 = 
 [-1.73205081]
u1 = 
 [[2.73205081]
 [0.        ]
 [1.        ]
 [1.        ]]
H1 = 
 [[-0.57735027  0.         -0.57735027 -0.57735027]
 [ 0.          1.          0.          0.        ]
 [-0.57735027  0.          0.78867513 -0.21132487]
 [-0.57735027  0.         -0.21132487  0.78867513]]


2. Step: Multiply the elementary reflector H1 from the left to the matrix A in order to zero out all the entries blow the first entry in the first column of A.

In [12]:
A2 = H1@A
print("A2= \n",A2)

A2= 
 [[-1.73205081e+00 -1.73205081e+00 -1.73205081e+00]
 [ 0.00000000e+00  2.00000000e+00 -1.00000000e+00]
 [-1.11022302e-16 -6.33974596e-01  1.46410162e+00]
 [-1.11022302e-16  2.36602540e+00  5.46410162e+00]]


3. Step: Do the first step for the second column of the new matrix A2.

In [13]:
# Get the second column of A2
a2 = A2[:,[1]]
# Set the first entry to zero, we do not want to change it
a2[0] = 0
# Calculate the norm
a2_norm = np.sqrt(sum(a2**2)) 
# Get alpha2, note here we look at a2[1]!
alpha2 = -np.sign(a2[1])*a2_norm
print("alpha2 = \n", alpha2)
# Construct e2
e2=np.zeros_like(a2)
e2[1] = 1
# and u2
u2 = a2 - alpha2*e2
print("u2 = \n", u2)
I = np.diag([1,1,1,1])
u2_norm = np.sqrt(sum(u2**2))
H2 = I - 2/u2_norm**2*u2@u2.T
print("H2 = \n", H2)

alpha2 = 
 [-3.16227766]
u2 = 
 [[ 0.        ]
 [ 5.16227766]
 [-0.6339746 ]
 [ 2.3660254 ]]
H2 = 
 [[ 1.          0.          0.          0.        ]
 [ 0.         -0.63245553  0.20048037 -0.74820293]
 [ 0.          0.20048037  0.97537919  0.09188612]
 [ 0.         -0.74820293  0.09188612  0.65707634]]


4. Step: Do step two with the elementary reflector H2 and the matrix A2.

In [14]:
A3=H2@A2
print("A3= ",A3)

A3=  [[-1.73205081e+00 -1.73205081e+00 -1.73205081e+00]
 [ 6.08094194e-17 -3.16227766e+00 -3.16227766e+00]
 [-1.18490252e-16 -3.95601797e-17  1.72964896e+00]
 [-8.31515368e-17  9.99706890e-17  4.47306545e+00]]


5. Step: Do the first step for the third column of the new matrix A3.

In [15]:
# Get the second column of A2
a3 = A3[:,[2]]
# Set the first entry to zero, we do not want to change it
a3[0] = 0
a3[1] = 0
# Calculate the norm
a3_norm = np.sqrt(sum(a3**2)) 
# Get alpha2, note here we look at a3[2]!
alpha3 = -np.sign(a3[2])*a3_norm
print("alpha3 = \n", alpha3)
# Construct e3
e3=np.zeros_like(a3)
e3[2] = 1
# and u3
u3 = a3 - alpha3*e3
print("u3 = \n", u3)
I = np.diag([1,1,1,1])
u3_norm = np.sqrt(sum(u3**2))
H3 = I - 2/u3_norm**2*u3@u3.T
print("H3 = \n", H3)

alpha3 = 
 [-4.79583152]
u3 = 
 [[0.        ]
 [0.        ]
 [6.52548048]
 [4.47306545]]
H3 = 
 [[ 1.          0.          0.          0.        ]
 [ 0.          1.          0.          0.        ]
 [ 0.          0.         -0.36065674 -0.93269862]
 [ 0.          0.         -0.93269862  0.36065674]]


6. Step: Construct R and Q

In [16]:
R=H3@A3
print("R= \n",R)

R= 
 [[-1.73205081e+00 -1.73205081e+00 -1.73205081e+00]
 [ 6.08094194e-17 -3.16227766e+00 -3.16227766e+00]
 [ 1.20289631e-16 -7.89748783e-17 -4.79583152e+00]
 [ 8.05265322e-17  7.29528277e-17 -8.90386688e-16]]


We see, up to numerical error, we have found an upper triangular matrix $R$. The orthogonal matrix $Q = H1\, H2\,H3$.

In [17]:
Q = H1@H2@H3
print("Q= \n",Q)

Q= 
 [[-0.57735027  0.31622777  0.62554324  0.41876284]
 [ 0.         -0.63245553  0.62554324 -0.45683219]
 [-0.57735027  0.31622777 -0.20851441 -0.72331764]
 [-0.57735027 -0.63245553 -0.41702883  0.3045548 ]]


In [18]:
print("QR = \n",Q@R)
print("A= \n",A)

QR = 
 [[ 1.00000000e+00  4.80113582e-16 -3.00000000e+00]
 [ 3.55543437e-33  2.00000000e+00 -1.00000000e+00]
 [ 1.00000000e+00  2.40620578e-16  1.00000000e+00]
 [ 1.00000000e+00  3.00000000e+00  5.00000000e+00]]
A= 
 [[ 1  0 -3]
 [ 0  2 -1]
 [ 1  0  1]
 [ 1  3  5]]


Hence we have found a QR decomposition of A via Householder transformations by hand. Now lets see how it works with with command scypi.qr(A)

In [19]:
import scipy.linalg as la

(q,r) = la.qr(A,mode='full')
print("q= \n",q)
print("r= \n",r)
print("qr= \n",q@r)
print("A= \n",A)

q= 
 [[-0.57735027  0.31622777  0.62554324  0.41876284]
 [-0.         -0.63245553  0.62554324 -0.45683219]
 [-0.57735027  0.31622777 -0.20851441 -0.72331764]
 [-0.57735027 -0.63245553 -0.41702883  0.3045548 ]]
r= 
 [[-1.73205081 -1.73205081 -1.73205081]
 [ 0.         -3.16227766 -3.16227766]
 [ 0.          0.         -4.79583152]
 [ 0.          0.          0.        ]]
qr= 
 [[ 1.00000000e+00  4.41068322e-16 -3.00000000e+00]
 [ 0.00000000e+00  2.00000000e+00 -1.00000000e+00]
 [ 1.00000000e+00  8.99849747e-17  1.00000000e+00]
 [ 1.00000000e+00  3.00000000e+00  5.00000000e+00]]
A= 
 [[ 1  0 -3]
 [ 0  2 -1]
 [ 1  0  1]
 [ 1  3  5]]


If we have by matrices the *full* mode might be to computational intensive. Therefore often the *economic* mode is used.

In [20]:
(q1,r1) = la.qr(A,mode='economic')
print("q1= \n",q1)
print("r1= \n",r1)
print("q1r1= \n",q1@r1)
print("A= \n",A)

q1= 
 [[-0.57735027  0.31622777  0.62554324]
 [-0.         -0.63245553  0.62554324]
 [-0.57735027  0.31622777 -0.20851441]
 [-0.57735027 -0.63245553 -0.41702883]]
r1= 
 [[-1.73205081 -1.73205081 -1.73205081]
 [ 0.         -3.16227766 -3.16227766]
 [ 0.          0.         -4.79583152]]
q1r1= 
 [[ 1.00000000e+00  4.41068322e-16 -3.00000000e+00]
 [ 0.00000000e+00  2.00000000e+00 -1.00000000e+00]
 [ 1.00000000e+00  8.99849747e-17  1.00000000e+00]
 [ 1.00000000e+00  3.00000000e+00  5.00000000e+00]]
A= 
 [[ 1  0 -3]
 [ 0  2 -1]
 [ 1  0  1]
 [ 1  3  5]]


With this mode we obtain matrices $q1$ and $r1$ that correspond to the matrices $Q_1$ and $R_1$ in the definition of QR decomposition in Patrick Walls notes.